In [1]:
from __future__ import print_function
#import utils.mnist_reader as um
import numpy as np
import matplotlib.pyplot as plt
#import mltools as ml
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import time
import numpy as np
from sklearn import metrics
import LoadData
%matplotlib auto

X_train, y_train, X_test, y_test = LoadData.Load()

Using matplotlib backend: Qt5Agg


In [2]:
#1. Data Visualizing
Xtr=X_train/255
Xte=X_test/255
plt.figure(figsize=(10,10))
for i in range(25):
    plt.ion()
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    img=np.reshape(Xtr[i],(28,28))
    plt.imshow(img,cmap=plt.cm.binary)

In [3]:
k_range = range(1,16,2)

k_accuracy_train = []

k_accuracy_test = []

for k in k_range:
    start_time = time.time()
    print('for k ========================= %f' % k)
    knn = KNeighborsClassifier(n_neighbors=k,n_jobs=8)
    # cross validation
    accuracy_train = np.mean(cross_val_score(knn, Xtr, y_train, cv=3,n_jobs=8,scoring='accuracy')) 
    k_accuracy_train.append(accuracy_train)
    knn.fit(Xtr,y_train)
    predict = knn.predict(Xte)
    accuracy_test = metrics.accuracy_score(y_test,predict)
    k_accuracy_test.append(accuracy_test)
    print('training took %fs!' % (time.time() - start_time))
    
plt.figure(1)
plt.plot(k_range,k_accuracy_train)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
plt.show()
plt.figure(2)
plt.plot(k_range,k_accuracy_test)
plt.xlabel('Value of K for KNN')
plt.ylabel('Test Accuracy')
plt.show()

for k ========================= 1.000000
training took 96.713411s!
for k ========================= 3.000000


KeyboardInterrupt: 

In [4]:
plt.figure(1)
plt.plot(k_range,k_accuracy_train)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
plt.show()
plt.figure(2)
plt.plot(k_range,k_accuracy_test)
plt.xlabel('Value of K for KNN')
plt.ylabel('Test Accuracy')
plt.show()

In [8]:
# From the plots above, I choose k=5
knn_uniform = KNeighborsClassifier(n_neighbors=5,weights='uniform',n_jobs=8)
knn_uniform.fit(Xtr,y_train)
predict_uniform = knn_uniform.predict(Xte)
accuracy_uniform = metrics.accuracy_score(y_test,predict_uniform)
knn_distance = KNeighborsClassifier(n_neighbors=5,weights='distance',n_jobs=8)
knn_distance.fit(Xtr,y_train)
predict_distance = knn_distance.predict(Xte)
accuracy_distance = metrics.accuracy_score(y_test,predict_distance)

print('For uniform, accuracy is %f'%accuracy_uniform)
print('For distance, accuracy is %f'%accuracy_distance)


For uniform, accuracy is 0.855400
For distance, accuracy is 0.857700


In [6]:
#If we use weights='distance', the accuracy will be better
#Next we will test various result analysis for each label
result=np.zeros((10,3))
pre=predict_distance
for i in range(0,10):
    TP,FN,FP=0,0,0
    for j in range(len(y_test)):
        if y_test[j]==i and pre[j]==i:
            TP+=1
        elif y_test[j]==i and pre[j]!=i:
            FN+=1
        elif pre[j]==i and y_test[j]!=i:
            FP+=1
    result[i][0]=TP/(TP+FP)
    result[i][1]=TP/(TP+FN)
    result[i][2]=2*result[i][0]*result[i][1]/(result[i][0]+result[i][1])
result=np.around(result,decimals=2)
print(" Precision  Recall  F-1 score")
for k in range(0,11):
    if k==10:
        print('avg',np.around(np.mean(result[:,0]),decimals=2),'   ',np.mean(result[:,1]),'   ',np.mean(result[:,2]))
    else:
        print(k,' ',result[k][0],'   ',result[k][1],'   ',result[k][2])
    

 Precision  Recall  F-1 score
0   0.81     0.83     0.82
1   0.99     0.97     0.98
2   0.75     0.8     0.77
3   0.9     0.87     0.89
4   0.79     0.77     0.78
5   1.0     0.83     0.91
6   0.63     0.63     0.63
7   0.89     0.96     0.92
8   0.98     0.96     0.97
9   0.9     0.97     0.93
avg 0.86     0.859     0.86


In [7]:
from plt_roc import *
plt_roc(knn_distance,Xte,y_test)
plt.ioff()